In [ ]:
import MLJ
import Flux
import CloudMicrophysics.PreprocessAerosolData as PAD
import CloudMicrophysics.ActivationEmulatorModels as AEM

In [ ]:
Standardizer = MLJ.@load Standardizer pkg = MLJModels verbosity = 0
NeuralNetworkRegressor = MLJ.@load NeuralNetworkRegressor pkg = MLJFlux verbosity = 0

In [ ]:
include("ReadAerosolDataset.jl")

In [ ]:
X_train, Y_train = read_aerosol_dataset("datasets/4modal_dataset1_train.csv")
X_train

In [ ]:
pipeline = PAD.preprocess_aerosol_data_with_ARG_act_frac |> Standardizer() |> MLJ.TransformedTargetModel(NeuralNetworkRegressor(
    builder = AEM.NNBuilder(50, 100, 30, 0.1, 0.3, 0.2),
    optimiser = Flux.Optimise.Adam(0.001, (0.9, 0.999), 1.0e-8),
    epochs = 200,
    loss = Flux.mse,
    batch_size = 50,
), transformer=PAD.target_transform, inverse=PAD.inverse_target_transform)
mach = MLJ.machine(pipeline, X_train, Y_train)
MLJ.fit!(mach, verbosity=2)
MLJ.save("emulators/4modal_nn_machine1.jls", mach)

In [ ]:
pipeline = PAD.preprocess_aerosol_data_with_ARG_act_frac |> Standardizer() |> MLJ.TransformedTargetModel(AEM.GPRegressor(1, 200, false, false, 0), transformer=PAD.target_transform, inverse=PAD.inverse_target_transform)
mach = MLJ.machine(pipeline, X_train, Y_train)
MLJ.fit!(mach, verbosity=2)
MLJ.save("emulators/1modal_gp_machine1.jls", mach)